In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DecimalType

# Creating a SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

In [2]:
df = spark.read.parquet('rail_data_cleaned_20230728132300.parquet')
df.limit(5).show()

+---------------+---------+------------------+-------------+--------------+-----------------+--------------------+----------------------+--------------------+----------------------+----+--------+------------+-------------------+---------------------+------------------+--------------------+-----------------+
|       route_id|unique_id|service_start_date|update_origin|train_platform|working_time_pass|working_time_arrival|working_time_departure|planned_time_arrival|planned_time_departure|pass|platform|train_length|actual_arrival_time|actual_departure_time|is_delayed_arrival|is_delayed_departure|__index_level_0__|
+---------------+---------+------------------+-------------+--------------+-----------------+--------------------+----------------------+--------------------+----------------------+----+--------+------------+-------------------+---------------------+------------------+--------------------+-----------------+
|202307277679020|   L79020|        2023-07-27|       Darwin|          STF

In [3]:
pandas_df = df.toPandas()
pandas_df

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_pass,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,pass,platform,train_length,actual_arrival_time,actual_departure_time,is_delayed_arrival,is_delayed_departure,__index_level_0__
0,202307277679020,L79020,2023-07-27,Darwin,STFD,NaT,2023-07-27 00:03:00,2023-07-27 00:03:30,NaT,NaT,NaN,5,None,2023-07-27 00:53:00,2023-07-27 00:53:00,True,True,8
1,202307277679020,L79020,2023-07-27,Darwin,SPNY205,NaT,2023-07-27 00:07:00,2023-07-27 00:13:30,NaT,NaT,NaN,None,None,2023-07-27 00:57:00,2023-07-27 00:58:00,True,True,10
2,202307277679020,L79020,2023-07-27,Darwin,PADTLL,NaT,2023-07-27 00:30:30,2023-07-27 00:31:00,NaT,NaT,NaN,B,None,2023-07-27 01:07:00,2023-07-27 01:07:00,True,True,16
3,202307277679020,L79020,2023-07-27,Darwin,WBRNPKS,NaT,2023-07-27 00:33:00,2023-07-27 00:35:00,NaT,NaT,NaN,A,None,2023-07-27 01:09:00,2023-07-27 01:10:00,True,True,17
4,202307277674553,L74553,2023-07-27,TD,SVNOAKS,NaT,2023-07-27 21:53:00,2023-07-27 21:54:30,2023-07-27 21:53:00,2023-07-27 21:54:00,NaN,1,8,NaT,NaT,False,False,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37095,202307278055183,P55183,2023-07-27,Darwin,BRADIN,NaT,2023-07-27 17:30:30,2023-07-27 17:33:30,2023-07-27 17:31:00,2023-07-27 17:33:00,NaN,4,None,2023-07-27 18:07:00,2023-07-27 18:08:00,True,True,94233
37096,202307278055183,P55183,2023-07-27,Darwin,HLFX,NaT,2023-07-27 17:44:00,2023-07-27 17:45:00,2023-07-27 17:44:00,2023-07-27 17:45:00,NaN,1,None,2023-07-27 18:18:00,2023-07-27 18:19:00,True,True,94235
37097,202307278055183,P55183,2023-07-27,Darwin,HBDNBDG,NaT,2023-07-27 17:58:30,2023-07-27 18:01:30,2023-07-27 17:59:00,2023-07-27 18:01:00,NaN,1,None,2023-07-27 18:29:00,2023-07-27 18:30:00,True,True,94238
37098,202307278055183,P55183,2023-07-27,Darwin,TODMRDN,NaT,2023-07-27 18:08:30,2023-07-27 18:09:30,2023-07-27 18:09:00,2023-07-27 18:09:00,NaN,1,None,2023-07-27 18:35:00,2023-07-27 18:35:00,True,True,94240


In [20]:
# print rows with respective datatypes
df.printSchema()

root
 |-- route_id: string (nullable = true)
 |-- unique_id: string (nullable = true)
 |-- service_start_date: string (nullable = true)
 |-- update_origin: string (nullable = true)
 |-- train_platform: string (nullable = true)
 |-- working_time_pass: timestamp_ntz (nullable = true)
 |-- working_time_arrival: timestamp_ntz (nullable = true)
 |-- working_time_departure: timestamp_ntz (nullable = true)
 |-- planned_time_arrival: timestamp_ntz (nullable = true)
 |-- planned_time_departure: timestamp_ntz (nullable = true)
 |-- pass: integer (nullable = true)
 |-- platform: string (nullable = true)
 |-- train_length: string (nullable = true)
 |-- actual_arrival_time: timestamp_ntz (nullable = true)
 |-- actual_departure_time: timestamp_ntz (nullable = true)
 |-- is_delayed_arrival: boolean (nullable = true)
 |-- is_delayed_departure: boolean (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [21]:
df.limit(5).show()

+---------------+---------+------------------+-------------+--------------+-----------------+--------------------+----------------------+--------------------+----------------------+----+--------+------------+-------------------+---------------------+------------------+--------------------+-----------------+
|       route_id|unique_id|service_start_date|update_origin|train_platform|working_time_pass|working_time_arrival|working_time_departure|planned_time_arrival|planned_time_departure|pass|platform|train_length|actual_arrival_time|actual_departure_time|is_delayed_arrival|is_delayed_departure|__index_level_0__|
+---------------+---------+------------------+-------------+--------------+-----------------+--------------------+----------------------+--------------------+----------------------+----+--------+------------+-------------------+---------------------+------------------+--------------------+-----------------+
|202307277674553|   L74553|        2023-07-27|           TD|       SVNOAK

In [8]:
# find duplicate rows
df.groupBy(df.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

+----------+
|sum(count)|
+----------+
|      NULL|
+----------+



In [25]:
Dict_Null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

{'route_id': 0,
 'unique_id': 0,
 'service_start_date': 0,
 'update_origin': 0,
 'train_platform': 0,
 'working_time_pass': 33004,
 'working_time_arrival': 0,
 'working_time_departure': 0,
 'planned_time_arrival': 52,
 'planned_time_departure': 0,
 'pass': 33004,
 'platform': 1314,
 'train_length': 23403,
 'actual_arrival_time': 4890,
 'actual_departure_time': 2306,
 'is_delayed_arrival': 0,
 'is_delayed_departure': 0,
 '__index_level_0__': 0}

In [24]:
df = df.dropna(subset='planned_time_departure')
df = df.dropna(subset='update_origin')
df.count()

33004

In [23]:
# get list of categorical var cols
categorical_columns = [c for c in df.columns if df.select(c).dtypes[0][1] == 'string']

# do a value counts for each categorical var
for col_name in categorical_columns:
    print(f"Unique values in {col_name}:")
    df.groupBy(col_name).count().show()

Unique values in route_id:
+---------------+-----+
|       route_id|count|
+---------------+-----+
|202307278024592|   40|
|202307278023407|    2|
|202307287142024|    8|
|202307277684886|    4|
|202307277623421|    3|
|202307277159156|    1|
|202307278931917|    2|
|202307278932995|   10|
|202307277679446|    4|
|202307288076176|    8|
|202307288075928|   36|
|202307278027471|    5|
|202307287195442|    1|
|202307278070469|    5|
|202307277683900|    1|
|202307278929733|    1|
|202307277683740|    2|
|202307287159171|    1|
|202307277194734|    1|
|202307278929658|    1|
+---------------+-----+
only showing top 20 rows

Unique values in unique_id:
+---------+-----+
|unique_id|count|
+---------+-----+
|   L79822|   11|
|   G59266|    1|
|   P25443|    2|
|   P24503|    9|
|   C18264|   31|
|   C16587|    1|
|   P71715|    3|
|   L34352|    2|
|   L23434|    4|
|   G95492|   10|
|   L23565|    1|
|   Y31409|   21|
|   Y30069|    1|
|   P27235|    5|
|   G67331|    1|
|   Y55093|    1|
|

In [28]:
# cast train length as int instead of str
df = df.withColumn('train_length', F.col('train_length').cast("int"))
df.printSchema()

root
 |-- route_id: string (nullable = true)
 |-- unique_id: string (nullable = true)
 |-- service_start_date: string (nullable = true)
 |-- update_origin: string (nullable = true)
 |-- train_platform: string (nullable = true)
 |-- working_time_pass: timestamp_ntz (nullable = true)
 |-- working_time_arrival: timestamp_ntz (nullable = true)
 |-- working_time_departure: timestamp_ntz (nullable = true)
 |-- planned_time_arrival: timestamp_ntz (nullable = true)
 |-- planned_time_departure: timestamp_ntz (nullable = true)
 |-- pass: integer (nullable = true)
 |-- platform: string (nullable = true)
 |-- train_length: integer (nullable = true)
 |-- actual_arrival_time: timestamp_ntz (nullable = true)
 |-- actual_departure_time: timestamp_ntz (nullable = true)
 |-- is_delayed_arrival: boolean (nullable = true)
 |-- is_delayed_departure: boolean (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [33]:
# import rail references and merge with api data
rail_ref = spark.read.csv('RailReferences.csv', header=True, inferSchema=True)
rail_ref.show()

+-----------+----------+-------+--------------------+---------------+--------+-------+--------+-------------------+--------------------+--------------+------------+
|   AtcoCode|TiplocCode|CrsCode|         StationName|StationNameLang|GridType|Easting|Northing|   CreationDateTime|ModificationDateTime|RevisionNumber|Modification|
+-----------+----------+-------+--------------------+---------------+--------+-------+--------+-------------------+--------------------+--------------+------------+
|9100PENZNCE|   PENZNCE|    PNZ|Penzance Rail Sta...|           NULL|       U| 147588|   30599|2003-11-04 00:00:00| 2011-09-30 14:47:28|             2|         rev|
| 9100STIVES|    STIVES|    SIV|St Ives (Cornwall...|           NULL|       U| 151947|   40127|2005-04-04 00:00:00| 2011-09-30 14:47:28|             4|         rev|
|9100CARBISB|   CARBISB|    CBB|Carbis Bay Rail S...|           NULL|       U| 152930|   38745|2003-11-04 00:00:00| 2011-09-30 14:47:28|             5|         rev|
| 9100STER

In [47]:
merged_df = df.join(rail_ref,df['train_platform'] == rail_ref['TiplocCode'],how='left')
merged_df.count()

33004

In [46]:
merged_df.select(F.col('StationName')).show()

+--------------------+
|         StationName|
+--------------------+
|Sevenoaks Rail St...|
|Stratford (London...|
|Maryland Rail Sta...|
|Forest Gate Rail ...|
|Manor Park Rail S...|
| Ilford Rail Station|
|Seven Kings Rail ...|
|Goodmayes Rail St...|
|Chadwell Heath Ra...|
|Romford Rail Station|
|Gidea Park Rail S...|
|Harold Wood Rail ...|
|Brentwood Rail St...|
|Birmingham Moor S...|
|Solihull Rail Sta...|
|Dorridge Rail Sta...|
|Warwick Parkway R...|
|Warwick Rail Station|
|Leamington Spa Ra...|
|Banbury Rail Station|
+--------------------+
only showing top 20 rows



In [49]:
merged_df.describe().show()

+-------+--------------------+---------+------------------+-------------+--------------+----+------------------+------------------+------------------+-----------+----------+-------+--------------------+---------------+--------+------------------+------------------+------------------+------------+
|summary|            route_id|unique_id|service_start_date|update_origin|train_platform|pass|          platform|      train_length| __index_level_0__|   AtcoCode|TiplocCode|CrsCode|         StationName|StationNameLang|GridType|           Easting|          Northing|    RevisionNumber|Modification|
+-------+--------------------+---------+------------------+-------------+--------------+----+------------------+------------------+------------------+-----------+----------+-------+--------------------+---------------+--------+------------------+------------------+------------------+------------+
|  count|               33004|    33004|             33004|        33004|         33004|   0|             

In [50]:
merged_df.groupBy('StationName').mean().limit(5).show()

+--------------------+---------+------------------+----------------------+----------------+-------------+-------------------+
|         StationName|avg(pass)| avg(train_length)|avg(__index_level_0__)|    avg(Easting)|avg(Northing)|avg(RevisionNumber)|
+--------------------+---------+------------------+----------------------+----------------+-------------+-------------------+
|Basingstoke Rail ...|     NULL|              NULL|     47780.73333333333|        463774.0|     152529.0|                2.0|
|Lea Bridge Rail S...|     NULL|               5.0|              25004.75|        536176.0|     187123.0|                1.0|
|Birkenhead Centra...|     NULL|              NULL|              44661.65|        332205.0|     388427.0|                3.0|
|Wimbledon Rail St...|     NULL|              NULL|     43079.27472527473|524810.934065934|     170666.0|  2.065934065934066|
|Manchester Oxford...|     NULL|3.7142857142857144|              48600.25|        384035.0|     397505.0|             

In [ ]:
merged_df.groupBy('')